In [1]:
import pandas as pd
import psycopg2
DBNAME = "opportunity_youth"
conn = psycopg2.connect(dbname=DBNAME)
cursor = conn.cursor()
cursor.execute("""SELECT table_name FROM information_schema.tables
       WHERE table_schema = 'public'""")
tables = []
for table in cursor.fetchall():
    tables.append(table[0])
tables

['pums_2017',
 'puma_names_2010',
 'wa_jobs_2017',
 'wa_geo_xwalk',
 'ct_puma_xwalk']

In [4]:
df_weight = pd.read_sql("""
SELECT A.puma, sporder, serialno, sch, schl, esr, cow, agep, pwgtp
FROM pums_2017 A
JOIN puma_names_2010 B ON A.puma = B.puma
WHERE state_name LIKE 'Washington%' AND puma_name LIKE 'King County%' AND puma_name LIKE '%South%'
AND (agep BETWEEN 16.0 AND 24.9)
""", conn)
df_weight

,puma,sporder,serialno,sch,schl,esr,cow,agep,pwgtp
0,11606,01,2013000003218,3,19,1,1,21.0,14.0
1,11606,02,2013000003218,1,16,1,1,21.0,15.0
2,11612,02,2013000007063,2,18,6,None,19.0,30.0
3,11613,02,2013000008046,2,13,6,None,17.0,36.0
4,11614,02,2013000011255,2,12,6,None,17.0,13.0
...,...,...,...,...,...,...,...,...,...
2873,11606,01,2017001526134,2,18,3,1,18.0,1.0
2874,11613,01,2017001530818,1,21,1,1,23.0,26.0
2875,11606,01,2017001490932,2,16,1,1,18.0,15.0
2876,11606,01,2017001491175,2,20,6,None,19.0,1.0


In [8]:
age_count = df_weight.groupby(by='agep').sum()['pwgtp']
esr_count = df_weight.groupby(by="esr").sum()["pwgtp"]
sch_count = df_weight.groupby(by="sch").sum()["pwgtp"]

pandas.core.series.Series

In [15]:
#Function that expands the table from 2878 rows to 68347 using pwgtp
def duplicate_rows(df, countcol):
    for _, row in df.iterrows():
        for i in range(int(row[countcol])-1):
            # Append this row at the end of the DataFrame
            df = df.append(row)

    # Remove countcol (could do a drop too to do that...)
    notcountcols = [x for x in df.columns if x != countcol]
    df = df[notcountcols]
    # optional: sort it by index
    df.sort_index(inplace=True)
    return df

In [61]:
#Runs the function that duplicates the table 
df_dup = duplicate_rows(df_weight, 'pwgtp')
df_dup

,puma,sporder,serialno,sch,schl,esr,cow,agep
0,11606,01,2013000003218,3,19,1,1,21.0
0,11606,01,2013000003218,3,19,1,1,21.0
0,11606,01,2013000003218,3,19,1,1,21.0
0,11606,01,2013000003218,3,19,1,1,21.0
0,11606,01,2013000003218,3,19,1,1,21.0
...,...,...,...,...,...,...,...,...
2875,11606,01,2017001490932,2,16,1,1,18.0
2875,11606,01,2017001490932,2,16,1,1,18.0
2875,11606,01,2017001490932,2,16,1,1,18.0
2876,11606,01,2017001491175,2,20,6,None,19.0


In [19]:
#Code to add Educational Groups
def school_range (schl):
    if int(schl) < 15: return "No diploma"
    elif int(schl) < 17: return "HS diploma or GED"
    elif int(schl) < 19: return "Some College, no degree"
    elif int(schl) < 25: return "Degree (Associate or higher)"
    else: return "Unknown"
df_dup["School_Level"]= df_dup.schl.apply(school_range)
df_dup

,puma,sporder,serialno,sch,schl,esr,cow,agep,School_Level
0,11606,01,2013000003218,3,19,1,1,21.0,Degree (Associate or higher)
0,11606,01,2013000003218,3,19,1,1,21.0,Degree (Associate or higher)
0,11606,01,2013000003218,3,19,1,1,21.0,Degree (Associate or higher)
0,11606,01,2013000003218,3,19,1,1,21.0,Degree (Associate or higher)
0,11606,01,2013000003218,3,19,1,1,21.0,Degree (Associate or higher)
...,...,...,...,...,...,...,...,...,...
2875,11606,01,2017001490932,2,16,1,1,18.0,HS diploma or GED
2875,11606,01,2017001490932,2,16,1,1,18.0,HS diploma or GED
2875,11606,01,2017001490932,2,16,1,1,18.0,HS diploma or GED
2876,11606,01,2017001491175,2,20,6,None,19.0,Degree (Associate or higher)


In [21]:
#Code to add OY_Status, for pivot grouping
def Y_Status (esr, sch):
    if (int(esr) == 3 or int(esr) == 6) and int(sch) == 1: return "Opportunity Youth"
    elif (int(esr) == 1 or int(esr) == 2 or int(esr) == 4 or int(esr) == 5) and int(sch) <= 15: return "Working without Diploma"
    else: return "Not Opportunity Youth"
df_dup["OY_Status"] = df_dup.apply(lambda x: Y_Status(x["esr"], x["sch"]), axis=1)

In [27]:
#Code that adds Age_Group and groups ages into bins
df_dup['Age_Group'] = pd.cut(x=df_dup['agep'], bins=[16, 18, 21, 24], labels=['16-18', '19-21', '21-24'])

In [23]:
#Groups OY_Status
puma_df_piv.groupby("OY_Status").count()

,puma,sporder,serialno,sch,schl,esr,cow,agep,pwgtp
OY_Status,,,,,,,,,
Not Opportunity Youth,1158,1158,1158,1158,1158,1158,390,1158,1158
Opportunity Youth,307,307,307,307,307,307,199,307,307
Working without Diploma,1413,1413,1413,1413,1413,1413,1413,1413,1413


In [47]:
#adds Total_Population_School for the second pivot
df_dup2 = df_dup
df_dup2 = df_dup2.reset_index()

In [36]:
#adds Total_Population_School for the second pivot
df_dup2 = df_dup
df_dup2["Total_Populations_Sch"]=df_dup2.groupby("Age_Group")["Age_Group"].transform("count")
#adds Total_Population column that takes the total population of each age group
df_dup2["Total_Populations"]=df_dup2.groupby("Age_Group")["Age_Group"].transform("count")
df_dup2["OY_Status_Counts"]=df_dup2.groupby("OY_Status")["OY_Status"].transform("count")

In [37]:
df_dup2.groupby("School_Level").count()

,puma,sporder,serialno,sch,schl,esr,cow,agep,OY_Status,Age_Group,Total_Populations_Sch,Total_Populations,OY_Status_Counts
School_Level,,,,,,,,,,,,,
Degree (Associate or higher),19902,19902,19902,19902,19902,19902,17540,19902,19902,19902,19902,19902,19902
HS diploma or GED,18882,18882,18882,18882,18882,18882,15767,18882,18882,18805,18805,18805,18882
No diploma,22374,22374,22374,22374,22374,22374,8721,22374,22374,14135,14135,14135,22374
"Some College, no degree",7279,7279,7279,7279,7279,7279,6235,7279,7279,7165,7165,7165,7279


In [38]:
df_dup2.groupby("Age_Group").count()

,puma,sporder,serialno,sch,schl,esr,cow,agep,School_Level,OY_Status,Total_Populations_Sch,Total_Populations,OY_Status_Counts
Age_Group,,,,,,,,,,,,,
16-18,16310,16310,16310,16310,16310,16310,8368,16310,16310,16310,16310,16310,16310
19-21,20710,20710,20710,20710,20710,20710,16761,20710,20710,20710,20710,20710,20710
21-24,22987,22987,22987,22987,22987,22987,21303,22987,22987,22987,22987,22987,22987


PIVOT TABLE CODE - School Level

In [63]:
df_dup2.sort_values(["School_Level"])
grouper = df_dup2.groupby(["Age_Group", "School_Level"])["Total_Populations_Sch"].count()
grouper = df_dup2.groupby(["Age_Group", "School_Level"], as_index=False).count()

Age_Group  School_Level                
16-18      Degree (Associate or higher)      755
           HS diploma or GED                3918
           No diploma                      10853
           Some College, no degree           784
19-21      Degree (Associate or higher)     7800
           HS diploma or GED                7547
           No diploma                       1887
           Some College, no degree          3476
21-24      Degree (Associate or higher)    11347
           HS diploma or GED                7340
           No diploma                       1395
           Some College, no degree          2905
Name: Total_Populations_Sch, dtype: int64
   Age_Group                  School_Level  index   puma  sporder  serialno  \
0      16-18  Degree (Associate or higher)    755    755      755       755   
1      16-18             HS diploma or GED   3918   3918     3918      3918   
2      16-18                    No diploma  10853  10853    10853     10853   
3      16-18  

In [66]:
#Creates School Level Pivot
sch_piv=grouper.pivot(index="School_Level", columns="Age_Group", values="Total_Populations")
sch_piv

Age_Group,16-18,19-21,21-24
School_Level,,,
Degree (Associate or higher),755,7800,11347
HS diploma or GED,3918,7547,7340
No diploma,10853,1887,1395
"Some College, no degree",784,3476,2905


PIVOT TABEL CODE - Opportunity Youth

In [58]:
df_dup2.sort_values(["Age_Group"])
grouper_oy = df_dup2.groupby(["Age_Group", "OY_Status"])["Total_Populations"].count()
grouper_oy = df_dup2.groupby(["Age_Group", "OY_Status"], as_index=False).count()

Age_Group  OY_Status              
16-18      Not Opportunity Youth      10298
           Opportunity Youth           1117
           Working without Diploma     4895
19-21      Not Opportunity Youth       5592
           Opportunity Youth           2769
           Working without Diploma    12349
21-24      Not Opportunity Youth       2363
           Opportunity Youth           3371
           Working without Diploma    17253
Name: Total_Populations, dtype: int64
  Age_Group                OY_Status  index   puma  sporder  serialno    sch  \
0     16-18    Not Opportunity Youth  10298  10298    10298     10298  10298   
1     16-18        Opportunity Youth   1117   1117     1117      1117   1117   
2     16-18  Working without Diploma   4895   4895     4895      4895   4895   
3     19-21    Not Opportunity Youth   5592   5592     5592      5592   5592   
4     19-21        Opportunity Youth   2769   2769     2769      2769   2769   
5     19-21  Working without Diploma  12349  12349 

In [65]:
#Creates OY_Status Pivot Table
oy_piv=grouper_oy.pivot(index="OY_Status", columns="Age_Group", values="Total_Populations")
oy_piv

Age_Group,16-18,19-21,21-24
OY_Status,,,
Not Opportunity Youth,10298,5592,2363
Opportunity Youth,1117,2769,3371
Working without Diploma,4895,12349,17253
